In [2]:
import pandas as pd
import numpy as np

In [3]:
cancer = pd.read_csv('../data/raw/string_cancer.tsv', sep='\t')
pre = pd.read_csv('../data/raw/string_preeclampsia.tsv', sep='\t')


rename_dict = {'#node1': 'node1', 'node1_string_id':'node1_id', 'node2_string_id':'node2_id',}
cancer = cancer.rename(columns=rename_dict)
pre = pre.rename(columns=rename_dict)

In [4]:
# load a xls file as pandas and get a specific tab
xls = pd.ExcelFile('../data/raw/proteomica.xlsx')
xls_cancer = pd.read_excel(xls, 'Cancer-Rat').drop('Nome da proteína (caso tenha que usar o nome do gene no string)', axis=1)
xls_pre = pd.read_excel(xls, 'Preeclampsia-Rat')
xls_cancer

,Query protein,Gene name,FC,Regulation
0,Q6AYZ1,3,-0.440370,Down
1,Q9EPH1,A1bg,-0.778146,Down
2,P06238,A2m,-0.711127,Down
3,G3V9J6,Abcb1b,-0.557322,Down
4,P68136,Acta1,-0.284675,Down
...,...,...,...,...
172,D4A3P1,Ubqln4,-0.528368,Down
173,D4ABI6,Uchl4,-0.634966,Down
174,Q9Z270,Vapa,-0.838787,Down
175,Q3MIE4,Vat1,-0.511388,Down


In [5]:
xls_cancer['Gene name'].nunique(), xls_pre['Gene name'].nunique()

(177, 324)

In [6]:
xls_pre['Gene name'].value_counts().sort_values(ascending=False)

Gene name
Gbe1              2
Slc2a1            1
Vcpip1 Vcip135    1
Utrn rCG_57340    1
Uso1 Vdp          1
                 ..
Gar1              1
Pappa2            1
Slc40a1           1
Agt Serpina8      1
Kng1              1
Name: count, Length: 324, dtype: int64

In [7]:
len(set(cancer.node1.unique().tolist() + cancer.node2.unique().tolist()))

103

In [8]:
len(set(pre.node1.unique().tolist() + pre.node2.unique().tolist()))

163

In [9]:
set(xls_cancer['Gene name'].unique().tolist()) - set(cancer.node1.unique().tolist() + cancer.node2.unique().tolist())

{3,
 'A1bg',
 'A2m',
 'Abcb1b',
 'Actg2',
 'Afm',
 'Aimp1',
 'Ak2',
 'Aldh2',
 'Alpp',
 'Anxa2',
 'Api5',
 'Atic',
 'Atp1b1',
 'Atp5b',
 'Atxn10',
 'Bag3',
 'Bsg',
 'Ca1',
 'Cand1',
 'Cdv3',
 'Cntrl',
 'Cotl1',
 'Cpsf6',
 'Csrp1',
 'Cyb5b',
 'Dbnl',
 'Ddost',
 'Ddx17',
 'Ddx3',
 'Dpysl2',
 'Dstn',
 'Eif5b',
 'Emc2',
 'En2',
 'Eno2',
 'Ero1l',
 'Etfb',
 'Fkbp14',
 'Gnas',
 'Gnpda1',
 'Gstm4',
 'Hax1',
 'Hba',
 'Hbb-b2',
 'Hpx',
 'Hspb1',
 'Itih2',
 'Krt18',
 'Lgals1',
 'Lonp1',
 'Mat2a',
 'Mug1',
 'Mug2',
 'Ndrg1',
 'Nefl',
 'Olfm4',
 'Pfkl',
 'Prdx4',
 'Prmt8',
 'Psap',
 'Ptbp1',
 'Rab33b',
 'Rab43',
 'Rack1',
 'Rap2c',
 'Rbp4',
 'Rilpl1',
 'Rpia',
 'Rragb',
 'Serpina3n',
 'Sfxn3',
 'Slc25a10',
 'Slc3a2',
 'Spata1',
 'Stk26',
 'Tgm2',
 'Tuba3a',
 'Tubb4b',
 'Tubb6',
 'Ubap2l',
 'Uchl4',
 'Vapa',
 'Vat1'}

In [10]:
set(xls_pre['Gene name'].unique().tolist()) - set(pre.node1.unique().tolist() + pre.node2.unique().tolist())

{'AABR07060872.1',
 'Abcb1a Abcb1 Abcb4',
 'Abhd12',
 'Abracl',
 'Adh1 Adh-1',
 'Afap1',
 'Agt Serpina8',
 'Aif1l',
 'Ak2 AK2',
 'Aldh2 Acad10',
 'Ankrd13a rCG_21176',
 'Aqp1 Chip28',
 'Arpp19',
 'Arsb',
 'Atp2b1',
 'Bgn',
 'Btf3 rCG_44623',
 'C4 C4a',
 'Camk2d',
 'Cand2',
 'Casp6',
 'Cavin1 Ptrf',
 'Cbr1l1',
 'Ccdc51',
 'Cd55 Daf1',
 'Cd5l Fcrl1',
 'Ceacam11 LOC292668 rCG_53839',
 'Ces1c Es2',
 'Chdh',
 'Ckap4',
 'Cmas',
 'Cnn3',
 'Cobll1',
 'Colec12 Clp1 Nsr2',
 'Cotl1 Clp',
 'Cpne1',
 'Creg1',
 'Creld2',
 'Crtap Crtapl1 LOC102555086',
 'Csrp2',
 'Ctla2a',
 'Ctsm',
 'Ctsql1 Ctsq LOC100364523',
 'Ctsql2',
 'Ctsr',
 'Cubn',
 'Cul7',
 'Dbnl Sh3p7',
 'Ddah1 Ddah',
 'Dnajb1 Dnajb1_predicted rCG_51112',
 'Dnajc3 P58ipk',
 'Dpp3',
 'ENSRNOG00000070933',
 'Efhd2 Sws1',
 'Emb Gp70',
 'Eml2 Emap2',
 'Epb41l3 Epb4.1l3',
 'Eps8l2',
 'Ero1a Ero1l Giig11',
 'Erp44',
 'Etfdh ETFDH',
 'Fam136a',
 'Fblim1',
 'Fbln1',
 'Fkbp5 rCG_60597',
 'Fndc3a',
 'Fnta',
 'Gab1',
 'Gar1',
 'Glg1',
 'Gnb4',
 'Grn',


In [11]:
xls_cancer.columns, xls_pre.columns

(Index(['Query protein', 'Gene name', 'FC', 'Regulation'], dtype='object'),
 Index(['Protein (UNIPROT)', 'protein_firstname', 'Gene name', 'P value',
        'Regulated', 'Fold change', 'Obs.'],
       dtype='object'))

In [12]:
xls_cancer = xls_cancer.rename(columns={'Gene name': 'gene', 'FC': 'fc'})
xls_cancer = xls_cancer.drop(['Regulation', 'Query protein'], axis=1)
# xls_cancer['fc'] = xls_cancer['fc'].str.replace(',', '.').astype(float)
xls_cancer['regulated'] = np.where(xls_cancer['fc'] > 0, 'up', 'down')
xls_cancer['cancer'] = 1
xls_cancer.columns

Index(['gene', 'fc', 'regulated', 'cancer'], dtype='object')

In [13]:
xls_pre = xls_pre.drop(['Protein (UNIPROT)', 'protein_firstname','P value','Obs.'], axis=1)
xls_pre = xls_pre.rename(columns={'Gene name': 'gene', 'Fold change': 'fc', 'Regulated': 'regulated'})
xls_pre['preeclampsia'] = 1
xls_pre['regulated'] = xls_pre['regulated'].str.lower()
# xls_pre['fc'] = xls_pre['fc'].str.replace(',', '.').astype(float)
xls_pre.columns

Index(['gene', 'regulated', 'fc', 'preeclampsia'], dtype='object')

In [14]:
xls_cancer.gene.nunique(), xls_pre.gene.nunique()

(177, 324)

Number of unique proteins in node1 and node2

In [15]:
cancer.node1.nunique(), cancer.node2.nunique()

(72, 71)

Genes that are present only in node1 or node2

In [16]:
diff = set(cancer.node1.unique()).symmetric_difference(set(cancer.node2.unique()))
len(diff)

63

Number of unique proteins(gene?)

In [17]:
len(set(cancer.node1.unique().tolist() + cancer.node2.unique().tolist()))

103

In [18]:
len(set(pre.node1.unique().tolist() + pre.node2.unique().tolist()))

163

In [19]:
f_df = xls_cancer.merge(xls_pre, on='gene', suffixes=('_cancer', '_pre'), how='outer')
f_df.cancer = f_df.cancer.fillna(0)
f_df.preeclampsia = f_df.preeclampsia.fillna(0)
f_df.to_csv('../data/processed/genes.csv', index=False)
f_df

,gene,fc_cancer,regulated_cancer,cancer,regulated_pre,fc_pre,preeclampsia
0,3,-0.440370,down,1.0,NaN,NaN,0.0
1,A1bg,-0.778146,down,1.0,NaN,NaN,0.0
2,A2m,-0.711127,down,1.0,NaN,NaN,0.0
3,Abcb1b,-0.557322,down,1.0,NaN,NaN,0.0
4,Acta1,-0.284675,down,1.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...
486,Casp6,NaN,NaN,0.0,up,0.559243,1.0
487,Gar1,NaN,NaN,0.0,up,0.580970,1.0
488,NaN,NaN,NaN,0.0,up,0.763275,1.0
489,Pappa2,NaN,NaN,0.0,up,0.811140,1.0


In [20]:
df_no_nans = f_df.dropna()
print(len(df_no_nans))
df_no_nans.sort_values('regulated_cancer', ascending=False)

12


,gene,fc_cancer,regulated_cancer,cancer,regulated_pre,fc_pre,preeclampsia
9,Actn1,0.240738,up,1.0,up,0.263762,1.0
52,Ehd4,0.576265,up,1.0,down,-0.293523,1.0
77,Hpx,0.444360,up,1.0,up,0.555551,1.0
170,Uba6,0.192763,up,1.0,down,-0.346560,1.0
48,Dnajb11,-0.428988,down,1.0,up,0.442867,1.0
51,Ehd1,-0.450791,down,1.0,down,-0.294619,1.0
61,F2,-0.553859,down,1.0,up,0.293442,1.0
90,Lgals1,-0.287505,down,1.0,up,0.315815,1.0
118,Prdx5,-0.360268,down,1.0,down,-0.353523,1.0
142,Ruvbl2,-0.296228,down,1.0,up,0.217608,1.0


In [21]:
cancer.to_csv('../data/processed/string_cancer.csv', index=False)
pre.to_csv('../data/processed/string_pre.csv', index=False)

### Adding a column that determines if the gene is present in both graphs
Requires the files already processed in Neo4J

In [31]:
graph = pd.read_csv('../data/processed/nodes2.csv')
graph['present_in'] = np.where(graph['n.cancer'] == 0, 'pre', np.where(graph['n.preeclampsia'] == 0, 'cancer', 'both'))
graph.to_csv('../data/processed/nodes2.csv', index=False)

In [33]:
graph

,n.name,n.cancer,n.preeclampsia,n.foldc_cancer,n.foldc_pre,n.regulated_cancer,n.regulated_pre,present_in
0,3,1,0,-0.440370,NaN,down,NaN,cancer
1,A1bg,1,0,-0.778146,NaN,down,NaN,cancer
2,A2m,1,0,-0.711127,NaN,down,NaN,cancer
3,Abcb1b,1,0,-0.557322,NaN,down,NaN,cancer
4,Acta1,1,0,-0.284675,NaN,down,NaN,cancer
...,...,...,...,...,...,...,...,...
484,Rpl34,0,1,NaN,0.400813,NaN,up,pre
485,Casp6,0,1,NaN,0.559243,NaN,up,pre
486,Gar1,0,1,NaN,0.580970,NaN,up,pre
487,Pappa2,0,1,NaN,0.811140,NaN,up,pre


In [32]:
graph.present_in.value_counts()

present_in
pre       312
cancer    165
both       12
Name: count, dtype: int64